In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
db = "20240418.nt"

In [ ]:
path_to_db = joinpath(ENV["SCRATCH"], "workspace", "blastdb", db)
# path_to_db = Mycelia.download_blast_db(db=db, source="ncbi")

In [ ]:
# compressed_fasta_export = Mycelia.export_blast_db(path_to_db = path_to_db)
compressed_fasta_export = path_to_db * ".fna.gz"

In [ ]:
@assert isfile(compressed_fasta_export)

In [ ]:
Mycelia.DEFAULT_MINIMAP_DENOMINATOR

In [ ]:
# Lawrencium LR3 specs
threads = 56
# 95% of 512Gb
mem_gb = 486
# mem_gb = mem_gb * .8 had OOM error
# mem_gb = mem_gb * 3/4 had OOM error
# cmd, outfile = Mycelia.minimap_index(fasta=compressed_fasta_export, mem_gb=Int(floor(mem_gb * 2/3)), threads=threads, mapping_type="sr", as_string=true)
# denominator 9 = OOM
cmd, outfile = Mycelia.minimap_index(fasta=compressed_fasta_export, mem_gb=mem_gb, threads=threads, mapping_type="sr", as_string=true, denominator=10)

In [ ]:
# Mycelia.add_bioconda_env("minimap2")

In [ ]:
# resubmit if we run out of time
if !isfile(outfile)
    println(outfile)
    Mycelia.lawrencium_sbatch(
        job_name=basename(outfile),
        mail_user="cameron.prybol@gmail.com",
        logdir=mkpath("$(homedir())/workspace/slurmlogs"),
        mem_gb=mem_gb,
        cpus_per_task=threads,
        partition="lr7",
        account="pc_mfnanofabio",
        time="3-00:00:00",
        qos="lr_normal",
        cmd=cmd)
end

In [ ]:
# try mapping something to it